In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('churn_analysis.csv')
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.churn = np.where(data.churn == 'True.', True, False)

- state — штат США

- account_length — длительность использования аккаунта

- area_code — деление пользователей на псевдорегионы, использующееся в телекоме

- intl_plan — подключена ли у пользователя услуга международного общения

- vmail_plan — подключена ли у пользователя услуга голосовых сообщений

- vmail_message — количество голосых сообщений, который пользователь отправил / принял

- day_calls — сколько пользователь совершил дневных звонков

- day_mins — сколько пользователь проговорил минут в течение дня

- day_charge — сколько пользователь заплатил за свою дневную активность

- eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности

- night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности

- intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения

- custserv_calls — сколько раз пользователь позвонил в службу поддержки

- treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)

- mes_estim — оценка интенсивности пользования интернет мессенджерами

- churn — результат оттока: перестал ли абонент пользоваться услугами оператора

In [3]:
data.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False


In [4]:
data.describe()

,account_length,area_code,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856,0.990999,0.484236
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491,0.819138,0.138560
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000,0.000000,0.390000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000,1.000000,0.480000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000,2.000000,0.580000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000,2.000000,0.960000


In [5]:
states = np.unique(data[data.treatment == 1].state.values)

In [6]:
pairs = []

for i in range(len(states)):
    for j in range(len(states)):
        if i >= j:
            continue
        pairs.append([states[i], states[j]])

In [7]:
pvalues = []

for pair in pairs:
    matrice = [[data[(data.state == pair[0]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[0]) & (data.churn == True) & (data.treatment == 1)].shape[0]],
               [data[(data.state == pair[1]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[1]) & (data.churn == True) & (data.treatment == 1)].shape[0]]]
    pvalues.append(stats.chi2_contingency(matrice, correction = False)[1])

In [8]:
len([p for p in pvalues if p < 0.05])

34

In [14]:
np.asarray(pvalues).mean()

0.5018273798739158

In [10]:
pvalues_true = []

for pair in pairs:
    matrice = [[data[(data.state == pair[0]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[0]) & (data.churn == True) & (data.treatment == 1)].shape[0]],
               [data[(data.state == pair[1]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[1]) & (data.churn == True) & (data.treatment == 1)].shape[0]]]
    pvalues_true.append(stats.chi2_contingency(matrice, correction = True)[1])

In [11]:
len([p for p in pvalues_true if p < 0.05])

0

In [15]:
np.asarray(pvalues_true).mean()

0.6640566382051047

In [16]:
pvalues_fisher = []

for pair in pairs:
    matrice = [[data[(data.state == pair[0]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[0]) & (data.churn == True) & (data.treatment == 1)].shape[0]],
               [data[(data.state == pair[1]) & (data.churn == False) & (data.treatment == 1)].shape[0],
                data[(data.state == pair[1]) & (data.churn == True) & (data.treatment == 1)].shape[0]]]
    pvalues_fisher.append(stats.fisher_exact(matrice)[1])

In [17]:
np.asarray(pvalues_fisher).mean()

0.6483383060020681

In [18]:
r, p = stats.pearsonr(data.day_calls, data.mes_estim)

In [20]:
print('Коэффициент корреляции:', r)
print('Достигаемый уровень значимость:', p)

Коэффициент корреляции: -0.05179435058757263
Достигаемый уровень значимость: 0.0027798836869738384


In [21]:
r, p = stats.spearmanr(data.day_calls, data.mes_estim)

In [22]:
print('Коэффициент корреляции:', r)
print('Достигаемый уровень значимость:', p)

Коэффициент корреляции: 0.043349880533927444
Достигаемый уровень значимость: 0.012317367189170541


In [23]:
def coefficient_v_kramer(contingency_matrix):
    chi2 = stats.chi2_contingency(contingency_matrix)[0]
    n = np.sum(np.sum(contingency_matrix))
    return np.sqrt(chi2 / (n * (min(contingency_matrix.shape) - 1)))

In [24]:
contingency_matrix = []

for state in states:
    state0 = data[(data.treatment == 1) & (data.state == state) & (data.churn == False)].shape[0]
    state1 = data[(data.treatment == 1) & (data.state == state) & (data.churn == True)].shape[0]
    contingency_matrix.append([state0, state1])

In [33]:
coef = coefficient_v_kramer(np.asarray(contingency_matrix))
print('Коэффициент корреляции Крамера:', coef)

Коэффициент корреляции Крамера: 0.2003932150203332


In [34]:
print('pvalue при проверке гипотезы о равенстве коэффициента корреляции Крамера нулю:',
      stats.chi2_contingency(np.asarray(contingency_matrix))[1])

pvalue при проверке гипотезы о равенстве коэффициента корреляции Крамера нулю: 0.7097590042778473


In [44]:
sample0 = np.where(data[data.treatment == 0].churn.values == True, 1, 0)
sample1 = np.where(data[data.treatment == 1].churn.values == True, 1, 0)
sample2 = np.where(data[data.treatment == 2].churn.values == True, 1, 0)

In [51]:
samples = [sample0, sample1, sample2]

In [47]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = sum(sample1) / n1
    p2 = sum(sample2) / n2 
    P = (p1 * n1 + p2 * n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1 / n1 + 1 / n2))

In [48]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [52]:
pvalues = []

for i, sample1 in enumerate(samples):
    for j, sample2 in enumerate(samples):
        if i >= j:
            continue
        pvalues.append(proportions_diff_z_test(proportions_diff_z_stat_ind(sample1, sample2)))

In [53]:
pvalues

[0.2283311639045107, 0.1564246886050802, 0.009348084294451109]

In [55]:
import statsmodels.stats.multitest as smm

In [56]:
reject, p_corrected, a1, a2 = smm.multipletests(pvalues, alpha = 0.05, method = 'holm') 

In [58]:
p_corrected

array([0.31284938, 0.31284938, 0.02804425])